In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.preprocessing import LabelEncoder

In [3]:
columns = [
    'movie_id', 'title', 'release_date', 'movie_popularity', 
    'vote_average', 'vote_count', 'budget', 'revenue', 'ratings', 'genres',
    'studios', 'actor_popularity_mean', 'director_popularity_mean'
]

target = ['revenue']

In [4]:
# Importing
# Load the data
file_path = Path('../Tables/ML_Test_pgs_checked.csv')
df = pd.read_csv(file_path, index_col=0)[:-2]
# , index_col=0
# df = df.loc[:, columns].copy()

# # Drop the null columns where all values are null
# df = df.dropna(axis='columns', how='all')

# # Drop the null rows
# df = df.dropna()

# Encoded Age Ratings
ratings_enc = {
    'G': 1,
    'PG': 2,
    'PG-13': 3,
    'R': 4,
    'NC-17': 5,
    'NR':6
}

df['ratings_enc'] = df['ratings'].apply(lambda x: ratings_enc[x])

df['release_month'] = pd.DatetimeIndex(df['release_date']).month
df.head()

,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_popularity_mean,director_popularity_mean,studios,genres,ratings_enc,release_month
movie_id,,,,,,,,,,,,,,
244,King Kong,1933-03-15,19.367,7.6,1017,672000,10000000,NR,1.102000,1.3365,['RKORadioPictures'],"['Adventure', 'Horror', 'ScienceFiction']",6,3
408,Snow White and the Seven Dwarfs,1938-02-04,76.361,7.1,5935,1488423,184925486,G,1.185000,1.3800,['WaltDisneyProductions'],"['Animation', 'Family', 'Fantasy']",1,2
630,The Wizard of Oz,1939-08-15,40.364,7.6,4340,2777000,33754967,G,1.233611,1.2040,['Metro-Goldwyn-Mayer'],"['Adventure', 'Family', 'Fantasy']",1,8
770,Gone with the Wind,1940-02-16,23.816,8.0,3115,4000000,402352579,G,1.045947,1.2040,"['Metro-Goldwyn-Mayer', 'Other']","['Drama', 'Romance', 'War']",1,2
10895,Pinocchio,1940-02-23,82.431,7.1,4632,2600000,84300000,G,1.116071,1.5470,"['RKORadioPictures', 'WaltDisneyProductions']","['Animation', 'Family']",1,2


In [5]:
def profitability_df(df):
    if (df['revenue'] >= df['budget']*2):
        return True
    else:
        return False

df['profitability'] = df.apply(profitability_df, axis = 1)
df

,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,actor_popularity_mean,director_popularity_mean,studios,genres,ratings_enc,release_month,profitability
movie_id,,,,,,,,,,,,,,,
244,King Kong,1933-03-15,19.367,7.6,1017,672000,10000000,NR,1.102000,1.3365,['RKORadioPictures'],"['Adventure', 'Horror', 'ScienceFiction']",6,3,True
408,Snow White and the Seven Dwarfs,1938-02-04,76.361,7.1,5935,1488423,184925486,G,1.185000,1.3800,['WaltDisneyProductions'],"['Animation', 'Family', 'Fantasy']",1,2,True
630,The Wizard of Oz,1939-08-15,40.364,7.6,4340,2777000,33754967,G,1.233611,1.2040,['Metro-Goldwyn-Mayer'],"['Adventure', 'Family', 'Fantasy']",1,8,True
770,Gone with the Wind,1940-02-16,23.816,8.0,3115,4000000,402352579,G,1.045947,1.2040,"['Metro-Goldwyn-Mayer', 'Other']","['Drama', 'Romance', 'War']",1,2,True
10895,Pinocchio,1940-02-23,82.431,7.1,4632,2600000,84300000,G,1.116071,1.5470,"['RKORadioPictures', 'WaltDisneyProductions']","['Animation', 'Family']",1,2,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610253,Halloween Kills,2021-10-14,434.457,6.9,1290,20000000,127000000,R,3.805955,2.7780,"['BlumhouseProductions', 'Miramax', 'Other', '...","['Horror', 'Thriller']",4,10,True
542178,The French Dispatch,2021-10-21,45.780,7.4,629,25000000,33000000,R,5.813298,6.1810,"['IndianPaintbrush', 'Other', 'StudioBabelsberg']","['Comedy', 'Drama', 'Romance']",4,10,False
576845,Last Night in Soho,2021-10-21,772.004,7.5,615,43000000,19000000,R,2.733038,6.8020,"['BigTalkProductions', 'Film4Productions', 'Fo...","['Horror', 'Mystery', 'Thriller']",4,10,False


In [6]:
# for test in df['studios']:
#     for studio in eval(test):
#         (studio)
        
# studio_revised = []
# for test in df['studios']:
#     holder = []
#     for studio in eval(test):
#         word = studio
#         if word not in holder:
#             holder.append(word)
#     studio_revised.append(holder)
    
# df.drop(['studios'],axis=1,inplace=True)
# df['studios'] = studio_revised

# df

In [7]:
# creating instance of labelencoder
labelencoder = LabelEncoder()

df['studios_cat'] = labelencoder.fit_transform(df.studios.values)
df['genres_cat'] = labelencoder.fit_transform(df.genres.values)

In [8]:
df = df.drop(['release_date', 'title', 'ratings', 'revenue'], axis=1)
df = df.drop(['studios', 'genres'], axis=1)
df.head()

,movie_popularity,vote_average,vote_count,budget,actor_popularity_mean,director_popularity_mean,ratings_enc,release_month,profitability,studios_cat,genres_cat
movie_id,,,,,,,,,,,
244,19.367,7.6,1017,672000,1.102000,1.3365,6,3,True,1183,220
408,76.361,7.1,5935,1488423,1.185000,1.3800,1,2,True,1225,241
630,40.364,7.6,4340,2777000,1.233611,1.2040,1,8,True,985,211
770,23.816,8.0,3115,4000000,1.045947,1.2040,1,2,True,983,346
10895,82.431,7.1,4632,2600000,1.116071,1.5470,1,2,True,1182,244


In [9]:
# Create our features
X = pd.get_dummies(df.drop(columns='profitability'))


# Create our target
y = pd.get_dummies(df['profitability'])

In [10]:
X.head()

,movie_popularity,vote_average,vote_count,budget,actor_popularity_mean,director_popularity_mean,ratings_enc,release_month,studios_cat,genres_cat
movie_id,,,,,,,,,,
244,19.367,7.6,1017,672000,1.102000,1.3365,6,3,1183,220
408,76.361,7.1,5935,1488423,1.185000,1.3800,1,2,1225,241
630,40.364,7.6,4340,2777000,1.233611,1.2040,1,8,985,211
770,23.816,8.0,3115,4000000,1.045947,1.2040,1,2,983,346
10895,82.431,7.1,4632,2600000,1.116071,1.5470,1,2,1182,244


In [11]:
# Check the balance of our target values
y = df['profitability']
y.value_counts()

True     1300
False     603
Name: profitability, dtype: int64

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({False: 464, True: 963})

# Balanced Random Forest Classifier

In [13]:
# Resample the training data with the BalancedRandomForestClassifier
from sklearn.preprocessing import StandardScaler
from imblearn.ensemble import BalancedRandomForestClassifier
from sklearn.datasets import make_classification

scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

brfc = BalancedRandomForestClassifier(n_estimators=100)
brfc.fit(X_train_scaled, y_train)

BalancedRandomForestClassifier()

In [14]:
# Calculated the balanced accuracy score
y_pred = brfc.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5

In [15]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[  0, 139],
       [  0, 337]], dtype=int64)

In [16]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.00      0.00      1.00      0.00      0.00      0.00       139
       True       0.71      1.00      0.00      0.83      0.00      0.00       337

avg / total       0.50      0.71      0.29      0.59      0.00      0.00       476



In [17]:
# List the features sorted in descending order by feature importance
sorted(zip(brfc.feature_importances_, X.columns), reverse=True)

[(0.2664383652733488, 'vote_count'),
 (0.10455378737230438, 'movie_popularity'),
 (0.10364608679291637, 'budget'),
 (0.09466432499166619, 'actor_popularity_mean'),
 (0.09386446690426942, 'genres_cat'),
 (0.09119848323959018, 'vote_average'),
 (0.0809101314718397, 'director_popularity_mean'),
 (0.07952931203040745, 'studios_cat'),
 (0.05224363617383457, 'release_month'),
 (0.03295140574982298, 'ratings_enc')]

# Easy Ensemble AdaBoost Classifier

In [18]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

eec = EasyEnsembleClassifier(n_estimators=300, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=300, random_state=1)

In [19]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.686751489016502

In [20]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

array([[101,  38],
       [119, 218]], dtype=int64)

In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.46      0.73      0.65      0.56      0.69      0.47       139
       True       0.85      0.65      0.73      0.74      0.69      0.47       337

avg / total       0.74      0.67      0.70      0.68      0.69      0.47       476



# Extra ML

In [22]:
# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [23]:
# Creating the decision tree classifier instance.
model = tree.DecisionTreeClassifier()
# Fitting the model.
model = model.fit(X_train_scaled, y_train)

In [24]:
# Making predictions using the testing data.
predictions = model.predict(X_test_scaled)

predictions

array([False,  True, False,  True,  True,  True,  True, False,  True,
       False,  True,  True,  True,  True,  True,  True,  True,  True,
        True, False,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True, False,  True,  True, False, False,
        True,  True,  True, False,  True, False, False,  True, False,
        True, False, False,  True,  True,  True,  True, False,  True,
        True,  True, False,  True,  True,  True,  True,  True,  True,
       False,  True,  True,  True,  True,  True,  True, False,  True,
       False,  True,  True, False,  True,  True, False,  True,  True,
       False,  True,  True, False,  True,  True,  True, False,  True,
        True, False,  True,  True,  True, False, False,  True, False,
        True,  True,  True, False, False,  True,  True,  True, False,
       False,  True,  True,  True,  True, False, False,  True,  True,
        True, False,  True,  True, False, False, False,  True,  True,
        True, False,